In [1]:
from src.nn.architectures.dag_encoder_decoder import EncoderDecoderDAG
import torch
from src.nn.services.decoding_strat import GreedyDAGStrategy

g:\Projects\Visual Studio Code\LM\.venv\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
embed_dim = 16
num_heads = 8
factor = -1
vocab_size = 5
max_len_enc = 10
max_len_dec = 40
num_enc_layers = 2
num_dec_layers = 2
dag_heads = 8
dag_phm_factor = -1
dag_lm_head_factor = -1

In [3]:
model = EncoderDecoderDAG(
    embed_dim=embed_dim,
    num_heads=num_heads,
    factor=factor,
    vocab_size=vocab_size,
    max_len_enc=max_len_enc,
    max_len_dec=max_len_dec,
    num_enc_layers=num_enc_layers,
    num_dec_layers=num_dec_layers,
    dag_heads=dag_heads,
    dag_phm_factor=dag_phm_factor,
    dag_lm_head_factor=dag_lm_head_factor
)

In [4]:
from src.datasets.utils.masking import self_attn_pad_mask, cross_attn_pad_mask, combine_masks, process_tokens

In [5]:
encoder_input = torch.tensor([
    [1, 1, 2, 0, 0],
    [1, 2, 3, 4, 0]
])

In [6]:
len_enc_not_pad, is_enc_not_pad = process_tokens(encoder_input, pad_idx=0)

In [7]:
enc_pad_mask = self_attn_pad_mask(is_enc_not_pad)

In [8]:
decoder_input = torch.tensor([
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1, -1],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
])

In [9]:
vertex_lens = torch.tensor([10, 12])

In [10]:
len_dec_not_pad, is_dec_not_pad = process_tokens(decoder_input, pad_idx=-1)

In [11]:
decoder_input[decoder_input == -1] = 0

In [12]:
dec_pad_mask = self_attn_pad_mask(is_dec_not_pad)

In [13]:
env_kv_dec_q_mask = cross_attn_pad_mask(is_enc_not_pad, is_dec_not_pad)

In [14]:
transitions, emissions = model(
    encoder_input,
    enc_pad_mask,
    decoder_input,
    dec_pad_mask,
    env_kv_dec_q_mask,
    vertex_lens
)

In [15]:
transitions.shape

torch.Size([2, 12, 12])

In [16]:
emissions.shape

torch.Size([2, 12, 5])

In [17]:
from unittest.mock import MagicMock

In [18]:
info = MagicMock()
info.pad_idx = -1

In [19]:
greedy = GreedyDAGStrategy(info=info,device="cpu")

In [20]:
output = greedy.decode(transitions, emissions)

g:\Projects\Visual Studio Code\LM\.venv\lib\site-packages\torch\nested\__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ..\aten\src\ATen\NestedTensorImpl.cpp:179.)
  return _nested.nested_tensor(


In [21]:
output

tensor([[ 2,  2,  2, -1, -1],
        [ 2,  2,  2,  2,  2]])